# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Лабораторная работа 6

### Форматирование строк

In [3]:
import pandas as pd
import numpy as np

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [4]:
recipes = pd.read_csv('data/recipes_sample.csv')

recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [5]:
sample = recipes.sample(n=5)
data = zip(sample.id, sample.minutes)
id_length = len(str(sorted(sample.id, key=lambda x: len(str(x)))[-1]))
minutes_length = len(str(sorted(sample.minutes, key=lambda x: len(str(x)))[-1]))

In [6]:
print(f"|{'id': ^{id_length+8}}|{'minutes': ^{minutes_length+8}}|")
print(f"|{'-':->{id_length + minutes_length + 17}}|")
for x, y in data:
    print(f"|{x: ^{id_length+8}}|{y: ^{minutes_length+8}}|")

|      id      |  minutes  |
|--------------------------|
|    144685    |    25     |
|    63479     |    365    |
|    379316    |    40     |
|    200989    |    30     |
|    178089    |    25     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [7]:
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [8]:
def show_info(name, steps, minutes, author_id):
    string = f'"{name.title()}"\n\n'
    for n, step in enumerate(steps, start=1):
        string += f"{n}. {step.capitalize()}\n"
    string += '----------\n'
    string += f"Автор: {author_id}\n"
    string += f"Среднее время приготовления: {minutes} минут\n"
    return string
    

In [9]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [10]:
import re
from bs4 import BeautifulSoup
pattern = r"([0-9]+[' '](\bhour\b|\bhours\b|\bminute\b|\bminutes\b))"
p = re.compile(pattern)

string = 'turn out onto a lightly floured board and knead for about 20 minutes'

res = p.findall(string)

res

[('20 minutes', 'minutes')]

In [11]:
with open('data/steps_sample.xml') as file:
    data = file.read()
    
soup = BeautifulSoup(data, 'xml')

In [12]:
recipes = soup.find_all('recipe')
for recipe in recipes:
    if recipe.id.text == '25082':
        exact = recipe
        break

In [13]:
steps = exact.find_all('step')


for step in steps:
    if p.findall(step.text):
        for x in p.findall(step.text):
            print(x[0], end=' ')
        print()


20 minutes 
10 minutes 
2 hours 
10 minutes 
20 minutes 30 minutes 


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [103]:
pattern = r"\A\bthis\b[0-9а-яА-ЯёЁa-zA-Z _]*[,]{1}\s*\bbut\b"
p  = re.compile(pattern)

string  = 'this is so trash, but'
res = p.match(string)

p.search(string)



<re.Match object; span=(0, 21), match='this is so trash, but'>

In [104]:
recipes = pd.read_csv('data/recipes_sample.csv')
descs = recipes.description.str.findall(pattern)
count = 0
        
for x in descs:
    try:
        if x[0]:
            count += 1
    except (IndexError, TypeError):
        pass
        
        
count

131

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [222]:
recipes = soup.find_all('recipe')
for recipe in recipes:
    if recipe.id.text == '72367':
        exact = recipe
        break

In [223]:
pattern = r"[0-9]+[' ']{1}[/]{1}[' ']{1}[0-9]+"
p = re.compile(pattern)

res = p.match('11 / 21')
print(res)



<re.Match object; span=(0, 7), match='11 / 21'>


In [241]:
steps = exact.steps
for step in steps:
    step = step.text
    if p.findall(step):
        substitute = p.findall(step)[0].split(' / ')
        substitute = '/'.join(substitute)
        step = re.sub(p, substitute, step)
        print(step)

mix butter , flour , 1/3 c
sugar and 1-1/4 t
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
combine apples , 1/3 c


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [246]:
from nltk.tokenize.toktok import ToktokTokenizer

steps = soup.find_all('step')

In [256]:
arr = list()

toktok = ToktokTokenizer()
for step in steps:
    words = toktok.tokenize(step)
    for word in words:
        if word.isalpha():
            arr.append(word.lower())
            
arr = np.array(arr)


In [257]:
len(np.unique(arr))

13524

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [259]:
from nltk.tokenize import sent_tokenize


arr = list()

recipes = pd.read_csv('data/recipes_sample.csv')
descs  = recipes.description 
for desc in descs:
    sentences = sent_tokenize(desc)
    arr.append(sentences)
    
arr.sort(key=lambda x: len(x))
sample = arr[:5]
sample

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/kir/nltk_data'
    - '/home/kir/anaconda3/nltk_data'
    - '/home/kir/anaconda3/share/nltk_data'
    - '/home/kir/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.
